# L2.4 Árboles de decisión

A lo largo de este curso estaremos trabajando con estas *Jupyter Notebooks*, tratando de poner en práctica los conceptos aprendidos en cada módulo. Pero, también las utilizaremos como herramienta de lectura para, antes de programar nuestro propio código, conocer cómo se usan algunas funciones, aprender sobre distintas librerías, etc.

En esta lectura generaremos árboles de decisión para un problema de clasificación.

Por favor no modifiques las celdas con las instrucciones, y solamente escribe código en las celdas donde así se te indica. **Si en algún momento seleccionas por error una celda de instrucciones y su apariencia cambia, simplemente presiona "Ctrl + Enter".**

Dentro de las celdas de código, las líneas que inician con un "\#" son comentarios y no se ejecutarán, simplemente sirven como instrucciones o descripciones útiles para ustedes.

En esta ocasión trabajaremos con la base de datos de Kaggle titulada *Kyphosis Dataset*, una base de datos que contiene 81 observaciones y 4 variables:
 - `Kyphosis`. absent o present, si se observa cifosis o no después de la operación.
 - `Age`. Edad, en meses.
 - `Number`. Cantidad de vértebras involucradas.
 - `Start`. Número de la vértebra más alta involucrada en la operación
 
Los datos fueron proporcionados originalmente por John M. Chambers y Trevor J. Hastle, y se descargaron directamente de [Kaggle](https://www.kaggle.com/datasets/abbasit/kyphosis-dataset).
 
Iniciemos como siempre, importando los datos a nuestro ambiente de trabajo. En esta ocasión el archivo lleva por nombre "L2.4 Kyphosis.csv". Adicionalmente, separa las variables de entrada de la variable de salida, designándolos como `X` y `y`.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    import pandas as pd<br>
    db = pd.read_csv("L2.4 Kyphosis.csv")<br>
    X = db.drop(['Kyphosis'],axis=1)<br>
    y = pd.get_dummies(db['Kyphosis'],drop_first=True)
</details>

In [1]:
# Importa pandas

# Lee el archivo

# Separa las entradas y la salida


Ahora, separemos nuestros datos en un set de entrenamiento y uno de prueba, utilizando una proporción de 85/15, y asegurándonos de que cada subconjunto tenga una proporción similar de clases en la variable de salida. Recuerda que puedes definir dicha condción usando el parámetro stratify de la función `train_test_split`. Asegúrate de que las proporciones sean las adecuadas imprimiendo la cantidad de muestras de cada clase para los datos originales, y para cada subconjunto.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    from sklearn.model_selection import train_test_split<br>
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y)<br>
    print(db.Kyphosis.value_counts())<br>
    print(y_train.value_counts())<br>
    print(y_test.value_counts())
</details>

In [11]:
# Importar la librería necesaria

# Separar los datos

# Imprimir las proporciones




Muy bien, llegó momento de generar y entrenr nuestro modelo. En esta ocasión volveremos a usar una función de sklearn, específicamente de la librería sklearn.tree. La función tiene por nombre `DecisionTreeClassifier`, pero te recomiendo que la importes como `DTC`. Trata de crear y entrenar el modelo en la misma línea de código. Obviamente, solo debemos usar los datos de entrenamiento.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    from sklearn.tree import DecisionTreeClassifier as DTC<br>
    tree = DTC().fit(X_train, y_train)
</details>

In [3]:
# Importar librería

# Crear y entrenar modelo


Inspeccionemos visualmente nuestro árbol. Importa la función `plot_tree` de sklearn.tree, y llámala dando como parámetro el modelo recién entrenado. Te recomiendo definir filled=True y feature_names con los nombres de tus variables, para que la gráfica sea más interpretable. Adicionalmente, te recomiendo agregar un punto y coma a esa línea de código para evitar desplegar el texto que esa función genera. También te aconsejo aumentar el tamaño de la gráfica a (15,10) con el parámetro figsize dentro de la función `plt.figure` (aunque el valor más adecuado dependerá de tu monitor).<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    import matplotlib.pyplot as plt<br>
    %matplotlib inline<br>
    from sklearn.tree import plot_tree<br>
    plt.figure(figsize=(15,10))<br>
    plot_tree(tree, filled=True, feature_names=X_train.columns);
</details>

In [10]:
# Importa la librería para generar gráficas

# Función mágica

# Importa plot_tree

# Define el tamaño de la figura

# Genera la gráfica del árbol de decisión


Revisemos ahora la calidad del modelo inicial en los datos de prueba, realizando una predicción en los mismos con nuestro modelo. Calcula tanto el accuracy como el f1-score, importando las funciones `accuracy_score` y `f1_score` de la librería sklearn.metrics. Imprime los resultados.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    from sklearn.metrics import accuracy_score, f1_score<br>
    yhat0 = tree.predict(X_test)<br>
    acc0 = accuracy_score(y_test, yhat0)<br>
    f10 = f1_score(y_test, yhat0)<br>
    print("Accuracy inicial:",acc0)<br>
    print("F1-score inicial:",f10)
</details>

In [12]:
# Importa las métricas

# Realiza la predicción

# Calcula accuracy

# Calcula f1

# Imprime resultados



Tratemos ahora de podar nuestro árbol, definiendo el valor adecuado de $\alpha$ usando una validación cruzada de 4 folds. Trataré de apoyarte paso a paso en este proceso; primero, importa numpy, así como las funciones `cross_val_score` y `StratifiedKFold` de sklearn.mode_selection. La primera función la usaremos para calcular un métrica de calida en un formato de validación cruzada. Sin embargo, dicha función no permite que la separación en *folds* se lleve a cabo de forma estratificada. Por lo mismo usaremos la segunda función para generar dicha separación.

Utiliza la función `StratifiedKFold` con el parámetro n_splits=4 para crear un sistema de separación de datos adecuado. Después, crea un vector de posibles valores para $\alpha$ con la función `linspace` de numpy, que genera secuencias numéricas con espaciado constante. A dicha función indícale un valor inicial (0.001), un valor final (0.2) y la cantidad de valores que quieres que se incluyan en la secuencia (entre 100 y 250). Inicializa un vector donde se almacenarán los resultados de cada *fold*.

Genera un ciclo for que recorra todos los valores del vector de posibles $\alpha$. En cada ciclo, genera un nuevo árbol, pero especificando el factor de corrección con el parámetro ccp_alpha. Después, calcula el f1-score y almacénalo en la variable previamente inicializada, promediando la salida de la función `cross_val_score`. Dicha función requiere que le indiques como parámetros el modelo que usará, los datos de entrenamiento (tanto `X` como `y`), el objeto de validación cruzada, y la métrica a utilizar. Asegúrate de usar el operador `append` para almacenar todos los resultados de las iteraciones.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    import numpy as np<br>
    from sklearn.model_selection import cross_val_score, StratifiedKFold<br>
    skf = StratifiedKFold(n_splits=4)<br>
    ccp = np.linspace(0.001, 0.2, 250)<br>
    cv_scores = []<br>
    for alpha in ccp:<br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;pruned_tree = DTC(ccp_alpha=alpha)<br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;cv_scores.append(np.mean(cross_val_score(pruned_tree, X_train, y_train, cv=skf, scoring='f1')))
</details>

In [13]:
# Importar numpy

# Importar funciones de validación cruzada

# Generar objeto de validación cruzada

# Generar secuencia de valores

# Inicializar variable de salida

# Ciclo donde se calcula el f1-score mediante validación cruzada




Excelente, revisemos ahora qué valor de alpha es óptimo para este sistema. Te recomiendo encontrar dicho valor usando la función `argmax` de numpy en el vector de resultados, y ubicar dentro del vector de posibles $\alpha$ el valor que corresponde a dicha posición. Imprime el resultado.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    alpha = ccp[np.argmax(cv_scores)]<br>
    print("Best alpha:",alpha)
</details>

In [14]:
# Encontrar el alpha óptimo

# Imprimir resultados


Ahora visualicemos nuestro árbol podado y revisemos que tantas o tan pocas hojas le quedaron.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    pruned_tree = DTC(ccp_alpha=alpha).fit(X_train, y_train)<br>
    plot_tree(pruned_tree, filled=True, feature_names=X_train.columns);
</details>

In [15]:
# Generar y entrenar el árbol podado

# Visualizar árbol


Analicemos ahora su calidad, usando las mismas métricas con que revisamos al árbol original.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    yhat_p = pruned_tree.predict(X_test)<br>
    acc_p = accuracy_score(y_test, yhat_p)<br>
    f1_p = f1_score(y_test, yhat_p)<br>
    print("Accuracy final:",acc_p)<br>
    print("F1-score final:",f1_p)
</details>

In [16]:
# Predicciones del nuevo modelo

# Accuracy

# F-1 score

# Imprimir resultados



Excelente trabajo, con esto damos por finalizada la lectura de esta sesión.